In [5]:
# 데이터 전처리
import pandas as pd

# 기계학습 모델
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, make_scorer

# 데이터 가져오기
train_data = pd.read_csv('data_train.csv')
test_data = pd.read_csv('data_test.csv')

# 타겟 변수 분할
X = train_data.drop('weather label', axis=1)
y = train_data['weather label']

# 결측치 처리
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
X_test_imputed = imputer.transform(test_data)

# 검증할 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# 그리드 서치 파라미터 설정
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
    'multi_class': ['multinomial']
}

# 로지스틱 회귀 모델 초기화
lr_model = LogisticRegression(max_iter=10000, random_state=42)

# 정의한 성능평가 지표
custom_scorer = make_scorer(lambda y_true, y_pred: accuracy_score(y_true, y_pred) + 2 * f1_score(y_true, y_pred, average='weighted'), greater_is_better=True)

# 그리드 서치 설정 - 3-fold 교차검증
grid_search = GridSearchCV(estimator=lr_model, param_grid=param_grid, scoring=custom_scorer, cv=3, n_jobs=-1)

# 그리드 서치 fit
grid_search.fit(X_train, y_train)

# 가장 좋은 파라미터와 점수
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# 점수들 확인
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# 모델 학습
best_lr_model = LogisticRegression(**best_params, max_iter=10000, random_state=42)
best_lr_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_test_pred = best_lr_model.predict(X_test_imputed)

# csv파일로 저장
submission_file = pd.DataFrame(y_test_pred, columns=['Predicted Label'])
submission_file.to_csv('2020.csv', index=False, header=False)


Best Parameters: {'C': 0.1, 'multi_class': 'multinomial', 'solver': 'lbfgs'}
Best Score: 2.3994616489206293
